# 일일 박스오피스 API를 이용
## 2018년01월11일 ~ 2018년11월3일까지의 개봉영화와 누적 관람객 수 찾기

In [2]:
import pandas as pd
import datetime
import requests

key1 = '93745210481f71d608b0ecd18fc8a7e9'

def crawl_data(start_date, end_date, keys=key1):
    final_list = []
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json'

    # 아래 Y,N,K,F가 다양성/상업, 국내/해외 영화를 각각 정의해준다.
    for single_date in pd.date_range(start_date, end_date):
        for multi, rep in zip(["N"], ["K"]):
            payload = {
                'key': keys,
                'targetDt': single_date.strftime('%Y%m%d'),
                'itemPerPage': '10',
                'multiMovieYn': multi,
                'repNationCd': rep
            }

            req = requests.get(url, params=payload)

            ## 여기서 리스트 형태로 저장하는 이유는 데이터프레임에 하나의 row로 인식되어 들어가게 하기 위함이다.
            for item in req.json()['boxOfficeResult']['dailyBoxOfficeList']:
                temp_list = []
                key_list = []
                for key, value in item.items():
                    key_list.append(key)
                    temp_list.append(value)

                temp_list.append(single_date)
                key_list.append('CurrentDate')

                temp_list.append(multi)
                key_list.append('multi')

                temp_list.append(rep)
                key_list.append('Nation')

                ## 위에서 temp_list에는 내용이, key_list에는 헤더가 담겼다.
                ## 이제 final_list에 내용을 담아준다. 사실 key_list는 한 번만 담아도 되지만 그냥 저렇게 두었다.
                final_list.append(temp_list)

    ## 마지막으로 final_list에 담긴 내용들을 데이터프레임으로 반환한다.
    return pd.DataFrame(final_list, columns=key_list)


movie = crawl_data("20180101", "20181130", keys=key1)



In [3]:
movie.head()

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,...,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,CurrentDate,multi,Nation
0,1,1,0,OLD,20150976,신과함께-죄와 벌,2017-12-20,7715346100,58.3,-852824100,...,76065842532,916689,-107268,-10.5,9456184,1644,7957,2018-01-01,N,K
1,2,2,0,OLD,20170590,1987,2017-12-27,4460740100,33.7,-224343200,...,19714744137,531520,-28526,-5.1,2473597,1097,4982,2018-01-01,N,K
2,3,3,0,OLD,20170402,강철비,2017-12-14,944300600,7.1,-48744300,...,32939198166,114310,-6536,-5.4,4128603,602,1451,2018-01-01,N,K
3,4,4,0,OLD,20174846,뽀로로 극장판 공룡섬 대모험,2017-12-07,109207600,0.8,60400,...,5958096200,13750,-3,0,787493,195,270,2018-01-01,N,K
4,5,5,1,OLD,20176769,광인옥한흠,2017-12-28,3171000,0.0,2273500,...,15236500,372,262,238.2,2009,11,12,2018-01-01,N,K


In [4]:
# 불필요한 열 데이터 삭제
del movie["rank"]
del movie['rnum']
del movie['rankInten']
del movie['rankOldAndNew']
del movie['salesInten']
del movie['salesChange']
del movie['audiInten']
del movie['audiChange']
del movie['salesShare']
del movie['salesAmt']

In [5]:
movie.head() # 열 삭제 확인

,movieCd,movieNm,openDt,salesAcc,audiCnt,audiAcc,scrnCnt,showCnt,CurrentDate,multi,Nation
0,20150976,신과함께-죄와 벌,2017-12-20,76065842532,916689,9456184,1644,7957,2018-01-01,N,K
1,20170590,1987,2017-12-27,19714744137,531520,2473597,1097,4982,2018-01-01,N,K
2,20170402,강철비,2017-12-14,32939198166,114310,4128603,602,1451,2018-01-01,N,K
3,20174846,뽀로로 극장판 공룡섬 대모험,2017-12-07,5958096200,13750,787493,195,270,2018-01-01,N,K
4,20176769,광인옥한흠,2017-12-28,15236500,372,2009,11,12,2018-01-01,N,K


- boxofficeType : 문자열	박스오피스 종류를 출력합니다.
- showRange	    : 문자열	박스오피스 조회 일자를 출력합니다.
- movieCd	    : 문자열	영화의 대표코드를 출력합니다. 
- movieNm	    : 문자열	영화명(국문)을 출력합니다.
- openDt	    : 문자열	영화의 개봉일을 출력합니다.
- salesAcc	    : 문자열	누적매출액을 출력합니다.
- audiCnt	    : 문자열	해당일의 관객수를 출력합니다. 
- audiAcc	    : 문자열	누적관객수를 출력합니다. 
- scrnCnt	    : 문자열	해당일자에 상영한 스크린수를 출력합니다. 
- showCnt	    : 문자열	해당일자에 상영된 횟수를 출력합니다.

In [8]:
movie[movie['movieNm'] == '1987']

,movieCd,movieNm,openDt,salesAcc,audiCnt,audiAcc,scrnCnt,showCnt,CurrentDate,multi,Nation
1,20170590,1987,2017-12-27,19714744137,531520,2473597,1097,4982,2018-01-01,N,K
8,20170590,1987,2017-12-27,21358190737,217249,2690846,1063,5044,2018-01-02,N,K
18,20170590,1987,2017-12-27,22687596737,174991,2865837,964,4231,2018-01-03,N,K
28,20170590,1987,2017-12-27,23928564037,163434,3029271,986,4260,2018-01-04,N,K
36,20170590,1987,2017-12-27,25636682734,202194,3231465,1012,4456,2018-01-05,N,K
43,20170590,1987,2017-12-27,29308960032,436764,3668229,1094,4812,2018-01-06,N,K
53,20170590,1987,2017-12-27,32848232932,421162,4089391,1067,4738,2018-01-07,N,K
60,20170590,1987,2017-12-27,34216684732,179977,4269368,1012,4545,2018-01-08,N,K
70,20170590,1987,2017-12-27,35549886032,175795,4445163,1052,4598,2018-01-09,N,K
80,20170590,1987,2017-12-27,36814820232,166737,4611900,1080,4666,2018-01-10,N,K


##### 아무래도 일일 영화데이터를 스크롤링한 것이므로 데이터가 중복되고 누적됨을 알 수 있음.
우리는 누적 관객 수만 구하면 되므로 마지막 아이만 뽑아야함

<hr>

# 최근 누적합계 찾기
##### 일간 데이터이므로 데이터 중복이 일어남.
##### 그래서 가장 최근의 누적 합계를 찾기 위해 쿼리 작성

In [10]:
# int형으로 묶어서 누적 구하기
movie.showCnt = movie.showCnt.astype(int)
movie.audiCnt = movie.audiCnt.astype(int)
movie.scrnCnt = movie.scrnCnt.astype(int)

In [11]:
movie_group = movie.groupby(['movieCd'])
recent = pd.DataFrame({
                    'movieNm' : movie_group['movieNm'].max(),
                    'screen' : movie_group['scrnCnt'].sum(),
                    'show' : movie_group['showCnt'].sum(),
                    'audience' : movie_group['audiAcc'].max(),
                    'sales' : movie_group['salesAcc'].max(),
                    'nation' : movie_group['Nation'].max(),
                    'openDate' : movie_group['openDt'].min()
                    }).reset_index()

recent

,movieCd,audience,movieNm,nation,openDate,sales,screen,show
0,19460021,494,자유만세,K,1946-10-21,988000,3,24
1,19588024,905,종각,K,,3045000,8,35
2,19598036,2095,여사장,K,,4175000,2,8
3,19608020,4112,이 생명 다하도록,K,,8234000,2,8
4,19618031,247,돼지꿈,K,,475000,1,3
5,19750082,1426,육체의 약속,K,1975-07-26,3990000,1,1
6,19760096,285,금욕,K,1976-03-13,943000,2,2
7,19780052,293,여기자 20년,K,1978-11-12,586000,2,8
8,19790046,3021,가을비 우산속에,K,1979-08-23,6040000,1,4
9,19830089,1318,얼굴이 아니고 마음입니다,K,1983-11-19,2619000,1,4


In [13]:
recent[recent['movieNm'] == '1987']

,movieCd,audience,movieNm,nation,openDate,sales,screen,show
86,20170590,7231830,1987,K,2017-12-27,58166910145,30678,110587


- 행이 하나만 나오고 다 합쳐짐

# 이상치 & 결측치 제거

- 재상영 영화를 제거하기 위해서 'movieCd'가 2018로 시작하는 row로만 구성
- 또한, 영화개봉일(openDate)이 공백인 것을 제외하기

In [14]:
temp = pd.DataFrame(columns=('movieCd', 'audience', 'movieNm', 'nation', 'openDate', 'sales', 'screen','show'))
movietemp = pd.DataFrame(columns=('movieCd', 'audience', 'movieNm', 'nation', 'openDate', 'sales', 'screen','show'))

#### 1. movieCd가 2018로 시작하는 아이만 movietemp 데이터프레임에 넣어준다

In [15]:

for i in range(0, len(recent)):
    if '2018' in recent.iloc[i].movieCd:
        temp = recent.iloc[i]
        movietemp = movietemp.append(temp)

In [16]:
movietemp.head()

,movieCd,audience,movieNm,nation,openDate,sales,screen,show
152,20180021,1,젊은 아내: 남편 교환의 날 (무삭제),K,2018-01-22,5000,1.0,1.0
153,20180063,846,라라,K,2018-02-22,6827500,48.0,53.0
154,20180085,123,초승달의 집,K,,615000,1.0,1.0
155,20180113,179,과외의 추억,K,2018-09-07,989000,20.0,21.0
156,20180141,147,정사 : 질투의 덫 - 감독판,K,2018-02-07,808000,22.0,22.0


- 값을 보면, movieCd가 2018로 시작하는 애로 구성되어 있는 것을 알 수 있다.
- index값이 섞여있으므로 정리해줘야함

In [17]:
movietemp = movietemp.reset_index(drop=True)
movietemp.head()

,movieCd,audience,movieNm,nation,openDate,sales,screen,show
0,20180021,1,젊은 아내: 남편 교환의 날 (무삭제),K,2018-01-22,5000,1.0,1.0
1,20180063,846,라라,K,2018-02-22,6827500,48.0,53.0
2,20180085,123,초승달의 집,K,,615000,1.0,1.0
3,20180113,179,과외의 추억,K,2018-09-07,989000,20.0,21.0
4,20180141,147,정사 : 질투의 덫 - 감독판,K,2018-02-07,808000,22.0,22.0


index 정렬됨

In [18]:
# 중복값 없음 확인
movietemp[movietemp['movieNm'] == '덕구']

,movieCd,audience,movieNm,nation,openDate,sales,screen,show
33,20181561,9453,덕구,K,2018-04-05,88620000,7696.0,19030.0


#### 2. openDate가 공백인 것 제외하고 다시 데이터프레임 구성

In [19]:
tmp = pd.DataFrame(columns=('movieCd', 'audience', 'movieNm', 'nation', 'openDate', 'sales', 'screen','show'))
final = pd.DataFrame(columns=('movieCd', 'audience', 'movieNm', 'nation', 'openDate', 'sales', 'screen','show'))

In [22]:
# openDate가 공백이 아닌 애만 제외 후 다시 저장
for i in range(0, len(movietemp)):
    if ' ' not in movietemp.iloc[i].openDate:
        tmp = movietemp.iloc[i]
        final = final.append(tmp)

In [23]:
final.head()

,movieCd,audience,movieNm,nation,openDate,sales,screen,show
0,20180021,1,젊은 아내: 남편 교환의 날 (무삭제),K,2018-01-22,5000,1.0,1.0
1,20180063,846,라라,K,2018-02-22,6827500,48.0,53.0
3,20180113,179,과외의 추억,K,2018-09-07,989000,20.0,21.0
4,20180141,147,정사 : 질투의 덫 - 감독판,K,2018-02-07,808000,22.0,22.0
5,20180142,146,전망 좋은 섹스-감독판,K,2018-01-24,800000,21.0,21.0


In [24]:
# 뒤에 빠져있을지 모르니 index 정렬
final = final.reset_index(drop=True)
final

,movieCd,audience,movieNm,nation,openDate,sales,screen,show
0,20180021,1,젊은 아내: 남편 교환의 날 (무삭제),K,2018-01-22,5000,1.0,1.0
1,20180063,846,라라,K,2018-02-22,6827500,48.0,53.0
2,20180113,179,과외의 추억,K,2018-09-07,989000,20.0,21.0
3,20180141,147,정사 : 질투의 덫 - 감독판,K,2018-02-07,808000,22.0,22.0
4,20180142,146,전망 좋은 섹스-감독판,K,2018-01-24,800000,21.0,21.0
5,20180225,431,동창회의 목적 3,K,2018-02-20,2290000,20.0,22.0
6,20180282,861,속닥속닥,K,2018-07-13,6912000,2516.0,8172.0
7,20180401,903,풀잎들,K,2018-10-25,7222550,601.0,1008.0
8,20180761,129,내 친구 엄마,K,2018-03-15,767000,20.0,20.0
9,20180782,128,착한 형수 3,K,2018-06-07,758000,20.0,20.0


In [28]:
# 공백값 없음 확인
final[final.openDate == ' ']

,movieCd,audience,movieNm,nation,openDate,sales,screen,show


In [30]:
final.to_csv("data1.csv", index=False)